In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import joblib
import re
from PIL import Image
# import dlib
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.client import device_lib
import sklearn
from sklearn.model_selection import train_test_split
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import random

In [226]:
os.chdir("D:/Down/inha_data (2)/inha_data")

In [2]:
model = tf.keras.models.load_model('D:/Down/facenet/model/facenet_keras.h5')

In [28]:
def getArr(img):    
    image = Image.open(img).resize((160,160))
    image = image.convert('RGB')
    pixels = np.asarray(image, dtype = 'float32')
    return pixels

In [55]:
test_left = os.listdir('test')[:6000]
test_right = os.listdir('test')[6000:]

p = re.compile("([0-9])")
tidx_left = list(map(lambda x: int("".join(p.findall(x))), test_left))
tidx_left = sorted(range(len(tidx_left)), key = lambda x :tidx_left[x])
test_left = [test_left[i] for i in tidx_left]

tidx_right = list(map(lambda x: int("".join(p.findall(x))), test_right))
tidx_right = sorted(range(len(tidx_right)), key = lambda x :tidx_right[x])
test_right = [test_right[i] for i in tidx_right]

testdat = pd.DataFrame({'left' : test_left,
                      'right' : test_right})

testdat.left = 'test/' + testdat.left
testdat.right = 'test/' + testdat.right

embtest = joblib.load('C:/Users/pc/OneDrive/문서/카카오톡 받은 파일/dataset_test.pkl')
idx1 = [False  if i in embtest['idx'] else True for i in testdat.index]

noface = testdat.loc[idx1,:].reset_index(drop=True)

no_test_left = []
for i in range(noface.shape[0]):
    temp = getArr(noface.iloc[i,0])
    no_test_left.append(temp)
    
no_test_left =np.array(no_test_left)

no_test_right = []
for i in range(noface.shape[0]):
    temp = getArr(noface.iloc[i,1])
    no_test_right.append(temp)
    
no_test_right =np.array(no_test_right)

In [80]:
def get_embedding(model, face_pixels):
    face_pixels = face_pixels.astype('int32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    samples = np.expand_dims(face_pixels, axis=0)
    yhat = model.predict(samples)
    return yhat[0]

In [116]:
no_test_left.shape

(89, 160, 160, 3)

In [125]:
left_emb = []
right_emb = []
for i in range(no_test_left.shape[0]):
    
    left_emb.insert(i, get_embedding(model, no_test_left[i]))
    right_emb.insert(i, get_embedding(model, no_test_right[i]))



In [149]:
diff = [left_emb[i] - right_emb[i] for i in range(len(left_emb))]

cos_sim = [(np.dot(left_emb[i], right_emb[i]))/(np.linalg.norm(left_emb[i])*np.linalg.norm(right_emb[i])) for i in range(len(left_emb))]

l1 = [np.linalg.norm(left_emb[i] - right_emb[i], ord =1) for i in range(len(left_emb))]

l2 = [np.linalg.norm(left_emb[i] - right_emb[i], ord =2) for i in range(len(left_emb))]

l_inf = [np.linalg.norm(left_emb[i] - right_emb[i], ord =np.inf) for i in range(len(left_emb))]

In [182]:
facenet_emb = np.array([np.hstack((cos_sim[i], diff[i], l1[i], l2[i], l_inf[i])) for i in range(len(diff))])

In [187]:
t1 = np.hstack([0,range(2,130),258,260,262])

t2 = np.hstack([1,range(130,258), 259,261,263])

In [223]:
no_idx = (np.arange(testdat.shape[0])[idx1]).tolist()

In [217]:
testdat2 = embtest['data']
testdat2_1 = testdat2[:, t1]
testdat2_2 = testdat2[:, t2]

fulltest_1 = pd.DataFrame(np.vstack((testdat2_1, facenet_emb)))
fulltest_2 = pd.DataFrame(np.vstack((testdat2_2, facenet_emb)))

fulltest_1['sortidx'] = np.array(embtest['idx'].tolist()+no_idx)
fulltest_2['sortidx'] = np.array(embtest['idx'].tolist()+no_idx)

fulltest_1 = fulltest_1.sort_values('sortidx', ascending=True).reset_index(drop=True).drop('sortidx', axis =1)
fulltest_2 = fulltest_2.sort_values('sortidx', ascending=True).reset_index(drop=True).drop('sortidx', axis =1)

In [225]:
joblib.dump(fulltest_1,'fulltest_1.pkl')
joblib.dump(fulltest_2,'fulltest_2.pkl')

['fulltest_1.pkl']

['fulltest_2.pkl']